### 3.0 LLMs and Chat Models

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

In [3]:
a = llm.predict("How many planets are there?")
b = llm.predict("How many planets are there?")

In [4]:
a, b

('As of current scientific understanding, there are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Please note that the status of Pluto as a planet is a matter of ongoing debate and is not universally accepted.',
 'There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

### 3.1 Predict Messages

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [6]:
messages = [
    SystemMessage(content='you are a geograph expert. and you only reply in Korea.'),
    AIMessage(content='와우! 제 이름은 딩고에요!'),
    HumanMessage(content="맥시코와 태국 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?"),
]

chat.predict_messages(messages)

AIMessage(content='멕시코와 태국 사이의 거리는 약 14,000km 정도 되요. 제 이름은 딩고에요!')

### 3.2 Prompt Templates

In [7]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

**3.2.1 PromptTemplate**

In [8]:
template = PromptTemplate.from_template("what is the distance between {country_a} and {country_b}",)

template.format(country_a="Mexico", country_b="Thailand")

'what is the distance between Mexico and Thailand'

In [9]:
vars(template)

{'input_variables': ['country_a', 'country_b'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'template': 'what is the distance between {country_a} and {country_b}',
 'template_format': 'f-string',
 'validate_template': False}

In [10]:
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

**3.2.2 ChatPromptTemplate**

In [18]:
messages = ChatPromptTemplate.from_messages([
    ("system", "you are a geograph expert. and you only reply in {language}.",),
    ("ai", "와우! 제 이름은 {name}에요!",),
    ("human", "{country_a}와 {country_b} 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?",),],
)

vars(messages)

{'input_variables': ['country_a', 'country_b', 'language', 'name'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='you are a geograph expert. and you only reply in {language}.')),
  AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='와우! 제 이름은 {name}에요!')),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country_a', 'country_b'], template='{country_a}와 {country_b} 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?'))],
 'validate_template': False}

In [21]:
prompt = messages.format_messages(
    language="Korea",
    name="딩고",
    country_a="Mexico",
    country_b="Thailand",
)

print(prompt)

[SystemMessage(content='you are a geograph expert. and you only reply in Korea.'), AIMessage(content='와우! 제 이름은 딩고에요!'), HumanMessage(content='Mexico와 Thailand 사이의 거리는 얼마인가. 그리고 너의 이름은 무엇이니?')]


In [20]:
chat.predict_messages(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 약 15,000km 정도 되요. 제 이름은 딩고에요!')

### 3.3 OutputParser and LCEL

**3.3.1 OutputParser**

In [27]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello, how, are, you?")

['Hello', 'how', 'are', 'you?']

In [35]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you ar asked will be answered with a comma seperated list of max {max_items} in lowercase. Do Not reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items = 10,
    question="What are the colors?"
)

print(prompt)

[SystemMessage(content='You are a list generating machine. Everything you ar asked will be answered with a comma seperated list of max 10 in lowercase. Do Not reply with anything else.'), HumanMessage(content='What are the colors?')]


In [36]:
chat.predict_messages(prompt)

AIMessage(content='red, blue, green, yellow, orange, purple, pink, black, white, brown')

In [39]:
result = chat.predict_messages(prompt)
p = CommaOutputParser()
parse_result = p.parse(result.content)
parse_result

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [40]:
type(parse_result)

list

**3.3.2 LCEL**

In [41]:
chain = template | chat | p

In [ ]:
vars(chain)

In [44]:
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

### 3.4 Chaining Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler(),],)

chef_promt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I wnat to cook {cuisine} food."),
])

chef_chain = chef_promt | chat

vars(chef_chain)

In [ ]:
veg_chef_promt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."
),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_promt | chat

vars(veg_chain)

In [ ]:
final_chain = {"recipe" : chef_chain} | veg_chain

vars(final_chain)

In [58]:
final_chain.invoke({
    "cuisine": "korean"
})

Great choice! How about making a classic Korean dish called Bibimbap? It's a delicious and colorful mixed rice bowl that's packed with flavor. Here's a simple recipe for you to try:

Ingredients:
- 1 cup of cooked white rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup of spinach
- 1 cup of bean sprouts
- 4 shiitake mushrooms, sliced
- 1/2 pound of beef (you can use ribeye or sirloin), thinly sliced
- 4 eggs
- Sesame oil
- Soy sauce
- Gochujang (Korean chili paste)
- Sesame seeds
- Salt and pepper
- Vegetable oil

Instructions:
1. Marinate the beef: In a bowl, mix the beef slices with 2 tablespoons of soy sauce, 1 tablespoon of sesame oil, and a pinch of black pepper. Let it marinate for at least 15 minutes.
2. Cook the rice according to package instructions and set aside.
3. In a pan, heat some vegetable oil over medium heat. Stir-fry the carrots, zucchini, and mushrooms separately until they are cooked but still slightly crunchy. Season each with a pinch of salt.
4. Blanch t

AIMessageChunk(content="For a vegetarian version of Bibimbap, you can easily substitute the beef with tofu or tempeh. Here's how you can prepare the tofu or tempeh as a replacement for the beef:\n\n**Tofu:**\n1. Use firm or extra-firm tofu for this recipe. Press the tofu to remove excess water by wrapping it in a clean kitchen towel and placing a heavy object on top for about 15-20 minutes.\n2. Cut the tofu into thin slices or cubes, depending on your preference.\n3. Marinate the tofu in a mixture of soy sauce, sesame oil, and a pinch of black pepper for at least 15 minutes.\n4. In a pan, heat some vegetable oil over medium heat and cook the marinated tofu until browned and slightly crispy.\n\n**Tempeh:**\n1. Cut the tempeh into thin slices or cubes.\n2. Steam the tempeh for about 10 minutes to remove any bitterness.\n3. Marinate the tempeh in a mixture of soy sauce, sesame oil, and a pinch of black pepper for at least 15 minutes.\n4. In a pan, heat some vegetable oil over medium heat 

## 4. MODEL IO

### 4.1 FewShotPromptTemplate

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, 
                  streaming=True, 
                  callbacks=[
                      StreamingStdOutCallbackHandler(),
                  ],
                )

In [2]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_promt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_promt,
    examples = examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Germany")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Germany?'

In [3]:
chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

### 4.2 FewShotChatMessagePromptTemplate 

In [26]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

### 4.3 LengthBasedExampleSelector

**4.3.1 LengthBasedExampleSelector**

In [38]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=200,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

print(prompt.format(country="Brazil"))

Human: What do you know about France?
AI: 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: What do you know about Italy?
AI: 
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


**4.3.2 Random Example Selector**

In [43]:
from typing import Any, Dict, List
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

In [47]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")
example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

print(prompt.format(country="Brazil"))

Human: What do you know about France?
AI: 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: What do you know about Brazil?


### 4.4 Serialization and Composition

In [49]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
prompt.format(country="Brazil")

'What is the capital of Brazil'

In [50]:
prompt = load_prompt("./prompt.yaml")
prompt.format(country="Brazil")

'What is the capital of Brazil'

In [56]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [("intro", intro), ("example", example), ("start", start),]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

print(full_prompt.format(
    character="Priate",
    example_question="What is your location?",
    example_answer="Arrrrg!! That is a secret!!!",
    question = "What is your favorite food?"
))


    
    You are a role playing assistant.
    And you are impersonating a Priate

                                     
    
    This is an example of how you talk:

    Human: What is your location?
    You: Arrrrg!! That is a secret!!!

                              
    
    Start now!

    Human: What is your favorite food?
    You:




In [57]:
chain = full_prompt | chat

chain.invoke({
    "character":"Priate",
    "example_question":"What is your location?",
    "example_answer":"Arrrrg!! That is a secret!!!",
    "question":"What is your favorite food?"
})

Arrrrg!! Me favorite food be a good ol' plate of rum-soaked sea biscuits! Aye, nothing beats the taste of salty biscuits washed down with a swig of me finest grog!

AIMessageChunk(content="Arrrrg!! Me favorite food be a good ol' plate of rum-soaked sea biscuits! Aye, nothing beats the taste of salty biscuits washed down with a swig of me finest grog!")

### 4.5 Caching

In [58]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

chat = ChatOpenAI(temperature=0.1)

chat.predict("How do you make italia pizza?")

'To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n\nInstructions:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or 260°C).\n\n2. Roll out your pizza dough on a floured surface to your desired thickness.\n\n3. Place the rolled out dough on a pizza stone or baking sheet.\n\n4. Spread a thin layer of tomato sauce over the dough, leaving a small border around the edges.\n\n5. Sprinkle a generous amount of shredded mozzarella cheese over the sauce.\n\n6. Tear fresh basil leaves and scatter them over the cheese.\n\n7. Drizzle a little olive oil over the top and season with salt and pepper.\n\n8. Place the pizza in the preheated oven and bake for about 10-15 minutes, or until the crust is golden brown and the cheese is melted and bubbly.\n\n9. Remove the pizza from the oven and let it cool slightly before slicing and serving.\n\nEnj

In [59]:
chat.predict("How do you make italia pizza?")

'To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n\nInstructions:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or 260°C).\n\n2. Roll out your pizza dough on a floured surface to your desired thickness.\n\n3. Place the rolled out dough on a pizza stone or baking sheet.\n\n4. Spread a thin layer of tomato sauce over the dough, leaving a small border around the edges.\n\n5. Sprinkle a generous amount of shredded mozzarella cheese over the sauce.\n\n6. Tear fresh basil leaves and scatter them over the cheese.\n\n7. Drizzle a little olive oil over the top and season with salt and pepper.\n\n8. Place the pizza in the preheated oven and bake for about 10-15 minutes, or until the crust is golden brown and the cheese is melted and bubbly.\n\n9. Remove the pizza from the oven and let it cool slightly before slicing and serving.\n\nEnj

In [61]:
from langchain.globals import set_debug

set_debug(True)

chat.predict("How do you make italia pizza?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italia pizza?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n\nInstructions:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or 260°C).\n\n2. Roll out your pizza dough on a floured surface to your desired thickness.\n\n3. Place the rolled out dough on a pizza stone or baking sheet.\n\n4. Spread a thin layer of tomato sauce over the dough, leaving a small border around the edges.\n\n5. Sprinkle a generous amount of shredded mozzarella cheese over the sauce.\n\n6. Tear fresh basil leaves and scatter them over the cheese.\n\n7. Drizzle a little olive oil over the top and season with salt and pepper.\n\n8. Pl

'To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n\nInstructions:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or 260°C).\n\n2. Roll out your pizza dough on a floured surface to your desired thickness.\n\n3. Place the rolled out dough on a pizza stone or baking sheet.\n\n4. Spread a thin layer of tomato sauce over the dough, leaving a small border around the edges.\n\n5. Sprinkle a generous amount of shredded mozzarella cheese over the sauce.\n\n6. Tear fresh basil leaves and scatter them over the cheese.\n\n7. Drizzle a little olive oil over the top and season with salt and pepper.\n\n8. Place the pizza in the preheated oven and bake for about 10-15 minutes, or until the crust is golden brown and the cheese is melted and bubbly.\n\n9. Remove the pizza from the oven and let it cool slightly before slicing and serving.\n\nEnj

In [62]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache('cache.db'))

chat.predict("How do you make italia pizza?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italia pizza?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [5.42s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough (you can make your own or buy pre-made dough)\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n- Toppings of your choice (such as pepperoni, mushrooms, bell peppers, etc.)\n\nHere is a basic recipe for making Italian pizza:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or higher).\n\n2. Roll out the pizza dough on a floured surface to your desired thickness. Place the dough on a pizza stone or baking sheet.\n\n3. Spread a thin layer of tomato sauce over the dough, leaving a border around the edges for the crust.\n\n4. Sprinkle the mozzarella cheese evenly over the sauce.\n\n5. Add your 

'To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough (you can make your own or buy pre-made dough)\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n- Toppings of your choice (such as pepperoni, mushrooms, bell peppers, etc.)\n\nHere is a basic recipe for making Italian pizza:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or higher).\n\n2. Roll out the pizza dough on a floured surface to your desired thickness. Place the dough on a pizza stone or baking sheet.\n\n3. Spread a thin layer of tomato sauce over the dough, leaving a border around the edges for the crust.\n\n4. Sprinkle the mozzarella cheese evenly over the sauce.\n\n5. Add your desired toppings on top of the cheese.\n\n6. Drizzle a little olive oil over the top of the pizza and season with salt and pepper.\n\n7. Bake the pizza in the preheated oven for about 10-15 minutes, or until the crust is golden brown and the c

In [64]:
chat.predict("How do you make italia pizza!?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italia pizza!?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough (you can make your own or buy pre-made dough)\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n- Toppings of your choice (such as pepperoni, mushrooms, bell peppers, etc.)\n\nHere is a basic recipe for making an Italian pizza:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or 260°C).\n\n2. Roll out your pizza dough on a floured surface to your desired thickness.\n\n3. Place the dough on a pizza stone or baking sheet lined with parchment paper.\n\n4. Spread a thin layer of tomato sauce over the dough, leaving a small border around the edges.\n\n5. Sprinkle a generous amount of shredded mozzar

'To make an Italian pizza, you will need the following ingredients:\n\n- Pizza dough (you can make your own or buy pre-made dough)\n- Tomato sauce\n- Mozzarella cheese\n- Fresh basil leaves\n- Olive oil\n- Salt and pepper\n- Toppings of your choice (such as pepperoni, mushrooms, bell peppers, etc.)\n\nHere is a basic recipe for making an Italian pizza:\n\n1. Preheat your oven to the highest temperature possible (usually around 500°F or 260°C).\n\n2. Roll out your pizza dough on a floured surface to your desired thickness.\n\n3. Place the dough on a pizza stone or baking sheet lined with parchment paper.\n\n4. Spread a thin layer of tomato sauce over the dough, leaving a small border around the edges.\n\n5. Sprinkle a generous amount of shredded mozzarella cheese over the sauce.\n\n6. Add your desired toppings on top of the cheese.\n\n7. Drizzle a little olive oil over the top of the pizza and season with salt and pepper.\n\n8. Bake the pizza in the preheated oven for about 10-15 minute

### 4.6 Serialization

**4.6.1 get_openai_callback**

In [65]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [4.03s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of water\n- 1 tablespoon of nuruk (fermentation starter)\n- 1 tablespoon of yeast\n\nInstructions:\n1. Rinse the rice thoroughly and soak it in water for at least 1 hour.\n2. Drain the rice and steam it until it is fully cooked.\n3. Let the rice cool down to room temperature.\n4. In a large bowl, mix the nuruk and yeast with the cooked rice.\n5. Cover the bowl with a clean cloth and let it ferment for 3-4 days in a warm place.\n6. After fermentation, strain the mixture through a cheesecloth to separate the liquid from the solids.\n7. Transfer the liquid to a clean container and let it sit for another 2-3 days to allow the flavors to develop.\n8. Your homemade soju is now ready to be enjoyed! Serve chilled and 

**4.6.2 Serialization**

In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model='gpt-3.5-turbo-16k',
)

chat.save("model.json")

In [67]:
from langchain.llms.loading import load_llm
chat = load_llm("model.json")
print(chat)

OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo-16k', 'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


/Users/sigi/hanwha_project/langchain_study/kang/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/sigi/hanwha_project/langchain_study/kang/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### 5.0 ConversationBufferMemory

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [5]:
memory.save_context({"input": "Hi!"}, {"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

### 5.1 ConversationBufferWindowMemory 

In [6]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [7]:
add_message(5,5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

### 5.2 ConversationSummaryMemory

In [2]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Kddorea is so pretty", "I wish I could go!!!")

In [3]:
get_history()

{'history': 'Nicolas from South Korea introduces himself to the AI, who responds enthusiastically. The human mentions how pretty South Korea is, prompting the AI to express a desire to visit.'}

### 5.3 ConversationSummaryBufferMemory

In [8]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [9]:
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [10]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
get_history()

{'history': [SystemMessage(content="The human introduces himself as Nicolas from South Korea. The AI responds by expressing admiration for Nicolas's location."),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

### 5.4 ConversationKGMemory

In [12]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.')]}

In [13]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does nicolas like"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.')]}

### 5.5 Memory on LLMChain

In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="chat_history",
)

template = """
    You are a chatbot that helps people with their daily life.

    {chat_history}
    Human: {question}
    AI:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a chatbot that helps people with their daily life.

    
    Human: My name is Nico
    AI:


> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [8]:
chain.predict(question="I`m living in seoul.")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a chatbot that helps people with their daily life.

    Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
    Human: I`m living in seoul.
    AI:


> Finished chain.


"That's great to hear! How are you finding living in Seoul? Is there anything specific you need help with in your daily life there?"

In [9]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a chatbot that helps people with their daily life.

    System: The human introduces themselves as Nico. The AI responds by saying it's nice to meet Nico and asks how it can assist them today.
Human: I`m living in seoul.
AI: That's great to hear! How are you finding living in Seoul? Is there anything specific you need help with in your daily life there?
    Human: what is my name?
    AI:


> Finished chain.


'Your name is Nico.'

In [10]:
print(memory.load_memory_variables({}))

{'chat_history': 'System: Nico introduces themselves and mentions they are living in Seoul. The AI responds positively and asks how they are finding living in Seoul, offering assistance with anything specific in their daily life there.\nHuman: what is my name?\nAI: Your name is Nico.'}


### 5.6 Chat Based Memory

In [11]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chatbot that helps people with their daily life."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot that helps people with their daily life.
Human: My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [12]:
chain.predict(question="I`m living in seoul.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot that helps people with their daily life.
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I`m living in seoul.

> Finished chain.


"That's great! How can I help you with living in Seoul? Do you have any specific questions or need information about the city?"

In [13]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot that helps people with their daily life.
System: The human introduces themselves as Nico. The AI responds by saying "Nice to meet you, Nico! How can I assist you today?"
Human: I`m living in seoul.
AI: That's great! How can I help you with living in Seoul? Do you have any specific questions or need information about the city?
Human: what is my name?

> Finished chain.


'Your name is Nico. How can I assist you further, Nico?'

### 5.7 LCEL Based Memory

In [33]:
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chatbot that helps people with their daily life."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(input):
    history = memory.load_memory_variables({})['history']
    print("input:" ,input)
    print("history: ", history)
    return history

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

chain.invoke({"question": "My name is Nico"})

input: {'question': 'My name is Nico'}
history:  []


AIMessage(content='Nice to meet you, Nico! How can I assist you today?')

In [34]:
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

invoke_chain("My name is Nico")

input: {'question': 'My name is Nico'}
history:  []
Nice to meet you, Nico! How can I assist you today?


In [35]:
invoke_chain("What is my name?")

input: {'question': 'What is my name?'}
history:  [HumanMessage(content='My name is Nico'), AIMessage(content='Nice to meet you, Nico! How can I assist you today?')]
Your name is Nico. How can I help you today, Nico?


### 6.1 Data Loaders and Splitters

**6.1.1 UnstructuredFileLoader**

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader

loader = UnstructuredFileLoader("./files/chapter_one.txt")

len(loader.load())

1

**6.1.2 RecursiveCharacterTextSplitter**

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

spliter = RecursiveCharacterTextSplitter(
    chunk_size=200,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
split_docs = loader.load_and_split(text_splitter=spliter)
print(split_docs[0])

page_content='Part 1, Chapter 1\n\nPart One\n\n1\n\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his' metadata={'source': './files/chapter_one.txt'}


In [4]:
spliter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
split_docs = loader.load_and_split(text_splitter=spliter)
print(split_docs[1])

page_content='breast in an effort to escape the vile wind, slipped quickly\n\nthrough the glass doors of Victory Mansions, though not\n\nquickly enough to prevent a swirl of gritty dust from entering along with him.' metadata={'source': './files/chapter_one.txt'}


**6.1.3 CharacterTextSplitter**

In [9]:
from langchain.text_splitter import CharacterTextSplitter

spliter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=200,
    chunk_overlap=50,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
split_docs = loader.load_and_split(text_splitter=spliter)
print(split_docs[1])

page_content='breast in an effort to escape the vile wind, slipped quickly\n\nthrough the glass doors of Victory Mansions, though not\n\nquickly enough to prevent a swirl of gritty dust from entering along with him.' metadata={'source': './files/chapter_one.txt'}


⛔️ **CharacterTextSplitter는 내 생각과 다르게 분할한다!**

In [24]:
from langchain.text_splitter import CharacterTextSplitter

# "\n\n"를 구분자로 설정하여 텍스트를 분할
splitter = CharacterTextSplitter(
    separator=r"\n\n",
    is_separator_regex=True,
    chunk_size=12,
    chunk_overlap=0,
)

# 예제 텍스트 설정
text = """Part 1, Chapter 1\n\nPart One\n\n1\n\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his"""

# 텍스트 분할 수행
split_texts = splitter.split_text(text)

# 분할된 청크의 수 출력
print(len(split_texts))

# 분할된 청크 출력
for i, chunk in enumerate(split_texts):
    print(f"Chunk {i + 1}:\n{chunk}\n")

Created a chunk of size 17, which is longer than the specified 12


4
Chunk 1:
Part 1, Chapter 1

Chunk 2:
Part One

Chunk 3:
1

Chunk 4:
It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his



In [25]:
from langchain.text_splitter import CharacterTextSplitter

# "\n\n"를 구분자로 설정하여 텍스트를 분할
splitter = CharacterTextSplitter(
    separator=r"\n\n",
    is_separator_regex=True,
    chunk_size=13,
    chunk_overlap=0,
)

# 예제 텍스트 설정
text = """Part 1, Chapter 1\n\nPart One\n\n1\n\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his"""

# 텍스트 분할 수행
split_texts = splitter.split_text(text)

# 분할된 청크의 수 출력
print(len(split_texts))

# 분할된 청크 출력
for i, chunk in enumerate(split_texts):
    print(f"Chunk {i + 1}:\n{chunk}\n")

Created a chunk of size 17, which is longer than the specified 13


3
Chunk 1:
Part 1, Chapter 1

Chunk 2:
Part One\n\n1

Chunk 3:
It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his



### 6.2 Tiktoken

In [10]:
spliter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=200,
    chunk_overlap=50,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
split_docs = loader.load_and_split(text_splitter=spliter)
print(split_docs[1])

page_content='no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut\n\noff during daylight hours. It was part of the economy drive\n\nin preparation for Hate Week. The flat was seven flights up,\n\nand Winston, who was thirty-nine and had a varicose ulcer\n\nabove his right ankle, went slowly, resting several times on\n\nthe way. On each landing, opposite the lift-shaft, the poster\n\nwith the enormous face gazed from the wall. It was one of\n\nthose pictures which are so contrived that the eyes follow\n\nyou about when you move. BIG BROTHER IS WATCHING\n\nYOU, the caption beneath it ran.\n\nInside the flat a fruity voice was reading out a list of fig\n\nures which had something to do with the production of\n\npig-iron. The voice came from an oblong metal plaque like' metadata={'source': './files/chapter_one.txt'}


### 6.4 Vector Store

**6.4.1 embedding**

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_query("hi")
vector

[-0.03091332069063776,
 -0.02041421168270496,
 -0.019505760491086016,
 -0.04178878834208551,
 -0.024813714510781345,
 0.024307577923883064,
 -0.017974369955820486,
 -0.01770183515712837,
 -0.0065019203189157744,
 -0.015910886872450965,
 0.025890880381560014,
 -0.006949656924423819,
 -0.017909480984128822,
 -0.011848808746080972,
 0.011465960646603284,
 0.016481912431040906,
 0.03875196137011493,
 0.0005187098496184129,
 0.032211107575051905,
 -0.00870167126729266,
 -0.01963553843446934,
 -0.004905640346297275,
 -0.009298654649733536,
 -0.014327584414774013,
 -0.022867032321514904,
 0.002483642527834438,
 0.010051371936763446,
 -0.011764452337823722,
 0.0026069326450696003,
 -0.026020658324943338,
 0.014535230241774468,
 0.0006987779917067696,
 -0.035767050045846224,
 -0.014963500807700843,
 -0.009486833971491013,
 -0.02474882553908968,
 0.006988590866232384,
 -0.021115018909968676,
 0.019181313769982482,
 -0.005687557856852956,
 0.0061288059377208794,
 -0.0007223004069264543,
 0.001407

In [30]:
len(vector)

1536

**6.4.2 VectorStore**

In [12]:
from langchain.vectorstores.chroma import Chroma

spliter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=200,
    chunk_overlap=50,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt") 
docs = loader.load_and_split(text_splitter=spliter)
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [13]:
result = vectorstore.similarity_search("where dose winston live")
print(result)

[Document(page_content='they could plug in your wire whenever they wanted to. You\n\nhad to live—did live, from habit that became instinct—in\n\nthe assumption that every sound you made was overheard,\n\nand, except in darkness, every movement scrutinized.\n\nWinston kept his back turned to the telescreen. It was\n\nsafer, though, as he well knew, even a back can be revealing.\n\nA kilometre away the Ministry of Truth, his place of work,\n\ntowered vast and white above the grimy landscape. This,\n\nhe thought with a sort of vague distaste—this was London,\n\nchief city of Airstrip One, itself the third most populous\n\nof the provinces of Oceania. He tried to squeeze out some\n\nchildhood memory that should tell him whether London\n\nhad always been quite like this. Were there always these vistas of rotting nineteenth-century houses, their sides shored', metadata={'source': './files/chapter_one.txt'}), Document(page_content='Part 1, Chapter 1\n\nPart One\n\n1\n\nIt was a bright cold da

In [14]:
len(result)

4

**6.4.3 Caching**

In [15]:
from langchain.storage.file_system import LocalFileStore
from langchain.embeddings.cache import CacheBackedEmbeddings

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = Chroma.from_documents(documents=docs, embedding=cached_embeddings)

### 6.6 RetrievalQA

In [18]:
from langchain.chains.retrieval_qa.base import RetrievalQA

llm = ChatOpenAI(temperature=0.1)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()  # 'retriver'를 'retriever'로 수정
)

chain.run("where does Winston live?")

'Winston lives in Victory Mansions.'